In [1]:
import duckdb
import pandas as pd
import torch
from transformers import *
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertModel, DistilBertTokenizer


2024-07-17 09:57:35.661648: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
comet_ml is installed but `COMET_API_KEY` is not set.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v4.40. Import as `from transformers import GenerationMixin` instead.
  warnings.w

In [2]:
#creating the connection to the duckdb file
duck_db = duckdb.connect('/Users/sheillapurwandiary/spurwand/BIPM/Semester2/2TWSM/OntologyProject/ontology/data/isrecon_AIS11.duckdb')
#filtering the ontology for level_3  research method and data analysis method.
ontology_df = duck_db.execute("""
SELECT * from main.ontology
where level_3  in ('research method','data analysis method')""").df()

#filtering the ontology for level_3  research method and data analysis method.

synonyms_df = duck_db.execute("""
SELECT  DISTINCT  synonym
FROM main.synonyms  s
join main.ontology o
on s.ent_id = o.ent_id 
where o.level_3  in ('research method', 'data analysis method')""").df()


In [3]:
ontology_df

,ent_id,definition,label,parent,level,related,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
0,research method,A research method is a systematic plan for con...,,methodological entity,3.0,None,entity,methodological entity,research method,,,,,,,
1,conceptual method,A conceptual method refers to a systematic app...,CONCEPTUAL_METHOD,research method,4.0,None,entity,methodological entity,research method,conceptual method,,,,,,
2,literature study,A literature study is a systematic and compreh...,CONCEPTUAL_METHOD,conceptual method,5.0,None,entity,methodological entity,research method,conceptual method,literature study,,,,,
3,structured literature research,Structured literature research is a methodical...,CONCEPTUAL_METHOD,literature study,6.0,None,entity,methodological entity,research method,conceptual method,literature study,structured literature research,,,,
4,literature sample,A literature sample refers to a specific piece...,CONCEPTUAL_METHOD,structured literature research,7.0,None,entity,methodological entity,research method,conceptual method,literature study,structured literature research,literature sample,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,questionable research practice,Questionable research practice refers to the a...,ANALYSIS_METHOD,quantitative analysis,5.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,,,,,
895,data dredging,Data dredging refers to the practice of conduc...,ANALYSIS_METHOD,questionable research practice,6.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,data dredging,,,,
896,p hacking,P hacking is the manipulative practice of rean...,ANALYSIS_METHOD,data dredging,7.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,data dredging,p hacking,,,
897,HARKing,HARKing refers to the practice of formulating ...,ANALYSIS_METHOD,questionable research practice,6.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,HARKing,,,,


In [4]:
synonyms_df

,synonym
0,atlasti
1,atlas.ti
2,QDA Miner Lite tool
3,video coded
4,coded all statement
...,...
31765,cohort polling
31766,panel survey
31767,skin conductance level
31768,skin conductance response


In [5]:
print('Before ', synonyms_df.shape)
synonyms_df.drop_duplicates(inplace=True)
print('After: ', synonyms_df.shape)

#resetting index to avoid issues
synonyms_df.reset_index(drop=True, inplace=True)

#  Creating a copy of the ontology_df and selecting the 'parent' column
data_ontology = ontology_df.copy()
data_ontology = data_ontology[['parent']]
data_ontology['text'] = data_ontology['parent']
del(data_ontology['parent'])

# Creating a copy of the synonyms_df and selecting the 'synonym' column
data_synonyms = synonyms_df.copy()
data_synonyms['text'] = data_synonyms['synonym']
del(data_synonyms['synonym'])

print('Before ', data_ontology.shape)
data_ontology.drop_duplicates(inplace=True)
print('After: ', data_ontology.shape)

#resetting index to avoid issues
data_ontology.reset_index(drop=True, inplace=True)
data_synonyms

Before  (31770, 1)
After:  (31770, 1)
Before  (899, 1)
After:  (220, 1)


,text
0,atlasti
1,atlas.ti
2,QDA Miner Lite tool
3,video coded
4,coded all statement
...,...
31765,cohort polling
31766,panel survey
31767,skin conductance level
31768,skin conductance response


In [6]:
data_ontology

,text
0,methodological entity
1,research method
2,conceptual method
3,literature study
4,structured literature research
...,...
215,statistical test assumption
216,homoskedasticity test
217,odds ratio homogeneity test
218,questionable research practice


In [7]:
data_synonyms_list = data_synonyms["text"].to_list()
data_ontology_list = data_ontology["text"].to_list()

In [10]:
# Load the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

loading file vocab.txt from cache at /Users/sheillapurwandiary/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/sheillapurwandiary/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading file tokenizer.json from cache at /Users/sheillapurwandiary/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading configuration file config.json from cache at /Users/sheillapurwandiary/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
   

In [19]:
def get_embeddings(term):
   # Tokenize the text
    tokens = tokenizer(term, padding=True, truncation=True, return_tensors="pt")
    # Generate embeddings
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Take the embeddings from the CLS token
    return embeddings

In [12]:
synonym_embeddings = {term: get_embeddings(term)[0] for term in data_synonyms_list}
ontology_embeddings = {term: get_embeddings(term)[0] for term in data_ontology_list}

In [20]:
def is_synonym(new_term):
    """
    Compare a new_term with the list of synonyms and ontology terms stored using cosine similarity.

    Args:
        new_term: A single term to be compared.

    Returns:
        A string indicating if the term is a synonym to any word in the list and the score, or if no synonym was found.
    """
    new_term_embds = get_embeddings([new_term])
    similarity_scores = {}
    overall_embeddings = {**synonym_embeddings, **ontology_embeddings}
    for term, embedding in overall_embeddings.items():
        similarity = cosine_similarity(new_term_embds[new_term].reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarity_scores[term] = similarity
    sorted_similarity_scores = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True))
    if sorted_similarity_scores and list(sorted_similarity_scores.values())[0] > 0.7:
        top_synonym = list(sorted_similarity_scores.keys())[0]
        return f"is synonym to {top_synonym} with a similarity score of {list(sorted_similarity_scores.values())[0]}"
    else:
        return "No close synonyms found in the list."

In [21]:
new_term = "apple"
is_synonym(new_term)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [17]:
def is_synonym(new_term):
    """
    Compare a new_term with the list of synonyms and ontology terms stored using cosine similarity.

    Args:
        new_term (str): a single term to be compared
    
    Returns:
        str: A string that tells if the term is a synonym to another word and the score, or if there was no synonym in the list.
    """
    if not isinstance(new_term, str):
        raise ValueError("new_term should be a string")
    
    # Get embeddings for the new term
    new_term_embd = get_embeddings(new_term)[0]
    
    similarity_scores = {}
    overall_embeddings = {**synonym_embeddings, **ontology_embeddings}
    # Calculate cosine similarity between the new term and each stored term
    for term, embedding in overall_embeddings.items():
        similarity = cosine_similarity(new_term_embd.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarity_scores[term] = similarity
    
    # Sort similarity scores in descending order
    sorted_similarity_scores = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)
    
    # Check if the highest similarity score is above the threshold (0.7)
    if sorted_similarity_scores and sorted_similarity_scores[0][1] > 0.7:
        top_synonym, top_score = sorted_similarity_scores[0]
        return f"{new_term} is a synonym to {top_synonym} with a similarity score of {top_score:.2f}"
    else:
        return f"{new_term} is not a close synonym to any term in the list."



In [18]:
new_term = "apple"
is_synonym(new_term)

'apple is a synonym to eyetracker with a similarity score of 0.97'